# Core Analysis: MARTA GTFS + Walk Isochrone

This notebook:
1. Downloads MARTA GTFS static data
2. Parses stops and identifies a sample downtown stop
3. Downloads Atlanta walking network using OSMnx
4. Creates a Pandana network for fast accessibility analysis
5. Computes a 500m walk isochrone from the sample stop
6. Exports GeoJSON files for visualization

**Author**: VIP Team 1270 - Smart Cities / Urban Systems  
**Project**: Atlanta SDG Portfolio Project


In [ ]:
# Core imports
import os
import sys
import zipfile
import requests
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union
import osmnx as ox
import pandana as pdna
import networkx as nx

print(f"OSMnx version: {ox.__version__}")
print(f"Pandana version: {pdna.__version__}")
print(f"GeoPandas version: {gpd.__version__}")


In [ ]:
# Set up paths - use relative paths from notebooks directory
BASE_DIR = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
if 'notebooks' in os.getcwd():
    os.chdir(BASE_DIR)
    
GTFS_DIR = os.path.join(BASE_DIR, 'gtfs_data')
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

# Create directories if they don't exist
for d in [GTFS_DIR, DATA_DIR, OUTPUT_DIR]:
    os.makedirs(d, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"GTFS directory: {GTFS_DIR}")
print(f"Output directory: {OUTPUT_DIR}")


## 1. Download MARTA GTFS Data


In [ ]:
# MARTA GTFS URL (static feed)
# Source: https://itsmarta.com/app-developer-resources.aspx
MARTA_GTFS_URL = "https://itsmarta.com/MARTA-GTFS.zip"
GTFS_ZIP_PATH = os.path.join(GTFS_DIR, 'google_transit.zip')

def download_gtfs(url, output_path, force=False):
    """Download GTFS zip file if not already present."""
    if os.path.exists(output_path) and not force:
        print(f"GTFS file already exists: {output_path}")
        return True
    
    print(f"Downloading MARTA GTFS from {url}...")
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        
        with open(output_path, 'wb') as f:
            f.write(response.content)
        
        print(f"Downloaded successfully: {len(response.content) / 1024:.1f} KB")
        return True
    except Exception as e:
        print(f"Error downloading GTFS: {e}")
        return False

# Download GTFS
download_gtfs(MARTA_GTFS_URL, GTFS_ZIP_PATH)


In [ ]:
# Extract GTFS zip
def extract_gtfs(zip_path, extract_dir):
    """Extract GTFS zip file."""
    stops_path = os.path.join(extract_dir, 'stops.txt')
    if os.path.exists(stops_path):
        print(f"GTFS already extracted to {extract_dir}")
        return True
    
    print(f"Extracting GTFS zip...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted to {extract_dir}")
        return True
    except Exception as e:
        print(f"Error extracting GTFS: {e}")
        return False

extract_gtfs(GTFS_ZIP_PATH, GTFS_DIR)

# List extracted files
print("\nExtracted files:")
for f in os.listdir(GTFS_DIR):
    print(f"  - {f}")


## 2. Parse MARTA Stops


In [ ]:
# Load stops.txt
stops_path = os.path.join(GTFS_DIR, 'stops.txt')
stops_df = pd.read_csv(stops_path)

print(f"Total MARTA stops: {len(stops_df)}")
print(f"\nColumns: {stops_df.columns.tolist()}")
stops_df.head()


In [ ]:
# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(stops_df['stop_lon'], stops_df['stop_lat'])]
stops_gdf = gpd.GeoDataFrame(stops_df, geometry=geometry, crs='EPSG:4326')

print(f"Stops GeoDataFrame created with {len(stops_gdf)} features")
print(f"CRS: {stops_gdf.crs}")
print(f"Bounds: {stops_gdf.total_bounds}")


In [ ]:
# Find the stop closest to downtown Atlanta (Five Points Station area)
# Downtown Atlanta coordinates: approximately 33.7537, -84.3901
DOWNTOWN_COORDS = Point(-84.3901, 33.7537)

# Calculate distance to downtown for each stop
stops_gdf['dist_to_downtown'] = stops_gdf.geometry.distance(DOWNTOWN_COORDS)

# Find closest stop
sample_stop = stops_gdf.loc[stops_gdf['dist_to_downtown'].idxmin()]

print(f"Sample stop (closest to downtown):")
print(f"  Name: {sample_stop['stop_name']}")
print(f"  ID: {sample_stop['stop_id']}")
print(f"  Lat: {sample_stop['stop_lat']:.6f}")
print(f"  Lon: {sample_stop['stop_lon']:.6f}")


In [ ]:
# Create a sample of stops around downtown for visualization
# Filter stops within ~3km of downtown
stops_gdf_proj = stops_gdf.to_crs('EPSG:32616')  # UTM zone 16N for Atlanta
downtown_proj = gpd.GeoSeries([DOWNTOWN_COORDS], crs='EPSG:4326').to_crs('EPSG:32616')[0]

stops_gdf_proj['dist_m'] = stops_gdf_proj.geometry.distance(downtown_proj)
downtown_stops = stops_gdf_proj[stops_gdf_proj['dist_m'] <= 3000].copy()
downtown_stops = downtown_stops.to_crs('EPSG:4326')

print(f"Stops within 3km of downtown: {len(downtown_stops)}")


## 3. Download Atlanta Walk Network (OSMnx)


In [ ]:
# Define bounding box around sample stop (approximately 2km buffer)
sample_lat = sample_stop['stop_lat']
sample_lon = sample_stop['stop_lon']

# Create a buffer for network download (about 1.5km in each direction)
BUFFER_DEG = 0.015  # Approximately 1.5km at Atlanta's latitude

north = sample_lat + BUFFER_DEG
south = sample_lat - BUFFER_DEG
east = sample_lon + BUFFER_DEG
west = sample_lon - BUFFER_DEG

print(f"Network bounding box:")
print(f"  North: {north:.4f}")
print(f"  South: {south:.4f}")
print(f"  East: {east:.4f}")
print(f"  West: {west:.4f}")


In [ ]:
# Download walk network from OSM
print("Downloading walk network from OpenStreetMap...")
print("(This may take 1-2 minutes)")

G = ox.graph_from_bbox(north, south, east, west, network_type='walk')

print(f"\nNetwork downloaded successfully!")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")


In [ ]:
# Convert to undirected graph for walk network
G_undirected = ox.convert.to_undirected(G)

# Get nodes and edges as GeoDataFrames
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G_undirected)

print(f"Undirected network:")
print(f"  Nodes: {len(nodes_gdf)}")
print(f"  Edges: {len(edges_gdf)}")


## 4. Create Pandana Network


In [ ]:
# Prepare data for Pandana
# Extract node coordinates
nodes_gdf = nodes_gdf.reset_index()
nodes_gdf['x'] = nodes_gdf.geometry.x
nodes_gdf['y'] = nodes_gdf.geometry.y

print(f"Node coordinates prepared")
nodes_gdf[['osmid', 'x', 'y']].head()


In [ ]:
# Prepare edges for Pandana
edges_gdf = edges_gdf.reset_index()

# Ensure we have 'length' column (in meters)
if 'length' not in edges_gdf.columns:
    # Calculate length from geometry
    edges_proj = edges_gdf.to_crs('EPSG:32616')
    edges_gdf['length'] = edges_proj.geometry.length

print(f"Edge lengths prepared")
print(f"Mean edge length: {edges_gdf['length'].mean():.1f} m")
print(f"Max edge length: {edges_gdf['length'].max():.1f} m")


In [ ]:
# Create Pandana network
print("Creating Pandana network...")

net = pdna.Network(
    nodes_gdf['x'],
    nodes_gdf['y'],
    edges_gdf['u'],
    edges_gdf['v'],
    edges_gdf[['length']],
    twoway=True
)

print(f"Pandana network created with {len(nodes_gdf)} nodes")


In [ ]:
# Precompute shortest paths for 500m and 1000m
print("Precomputing shortest paths...")
net.precompute(1000)  # Precompute up to 1000m
print("Precomputation complete!")


## 5. Compute 500m Walk Isochrone


In [ ]:
# Find the network node closest to our sample stop
origin_node_id = net.get_node_ids([sample_lon], [sample_lat])[0]

print(f"Sample stop: {sample_stop['stop_name']}")
print(f"Origin node ID: {origin_node_id}")

# Get the coordinates of the origin node
origin_node = nodes_gdf[nodes_gdf['osmid'] == origin_node_id].iloc[0]
print(f"Origin node coords: ({origin_node['x']:.6f}, {origin_node['y']:.6f})")


In [ ]:
# Compute distances from origin to all nodes using Pandana
MAX_DISTANCE = 500  # meters

# Set the origin point
net.set_pois('origin', MAX_DISTANCE * 2, 1, 
             x_col=pd.Series([sample_lon]), 
             y_col=pd.Series([sample_lat]))

# Get distances to nearest POI (our origin)
distances = net.nearest_pois(MAX_DISTANCE * 2, 'origin', num_pois=1)
distances.columns = ['distance']

print(f"Distances computed for {len(distances)} nodes")
print(f"Nodes within {MAX_DISTANCE}m: {(distances['distance'] <= MAX_DISTANCE).sum()}")


In [ ]:
# Get reachable nodes (within 500m)
reachable_mask = distances['distance'] <= MAX_DISTANCE
reachable_node_ids = distances[reachable_mask].index.tolist()

print(f"Reachable nodes within {MAX_DISTANCE}m: {len(reachable_node_ids)}")

# Get coordinates of reachable nodes
reachable_nodes = nodes_gdf[nodes_gdf['osmid'].isin(reachable_node_ids)]
print(f"Reachable nodes GeoDataFrame: {len(reachable_nodes)} features")


In [ ]:
# Create isochrone polygon from reachable nodes
# Using convex hull for simplicity (alpha shape would be more accurate but more complex)

if len(reachable_nodes) > 2:
    # Create points from reachable nodes
    points = reachable_nodes.geometry.tolist()
    
    # Create convex hull
    isochrone_polygon = unary_union(points).convex_hull
    
    print(f"Isochrone polygon created")
    print(f"  Type: {isochrone_polygon.geom_type}")
    print(f"  Area: {isochrone_polygon.area:.8f} sq degrees")
else:
    print("Not enough nodes to create isochrone polygon")
    isochrone_polygon = None


In [ ]:
# Create GeoDataFrame for isochrone
isochrone_gdf = gpd.GeoDataFrame(
    {
        'name': [f'{MAX_DISTANCE}m Walk Isochrone'],
        'origin_stop': [sample_stop['stop_name']],
        'distance_m': [MAX_DISTANCE],
        'node_count': [len(reachable_nodes)],
        'origin_lat': [sample_lat],
        'origin_lon': [sample_lon]
    },
    geometry=[isochrone_polygon],
    crs='EPSG:4326'
)

print("Isochrone GeoDataFrame created:")
isochrone_gdf


## 6. Export GeoJSON Files


In [ ]:
# Export isochrone polygon
isochrone_path = os.path.join(OUTPUT_DIR, 'isochrone_500m.geojson')
isochrone_gdf.to_file(isochrone_path, driver='GeoJSON')
print(f"Exported: {isochrone_path}")

# Verify
verify_gdf = gpd.read_file(isochrone_path)
print(f"Verification - loaded {len(verify_gdf)} features")


In [ ]:
# Export sample MARTA stops (downtown area)
# Select key columns for export
export_cols = ['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'geometry']
available_cols = [c for c in export_cols if c in downtown_stops.columns]
stops_export = downtown_stops[available_cols].copy()

stops_path = os.path.join(OUTPUT_DIR, 'marta_stops_sample.geojson')
stops_export.to_file(stops_path, driver='GeoJSON')
print(f"Exported: {stops_path}")
print(f"  Features: {len(stops_export)}")


In [ ]:
# Export origin point (sample stop)
origin_gdf = gpd.GeoDataFrame(
    {
        'stop_name': [sample_stop['stop_name']],
        'stop_id': [sample_stop['stop_id']],
        'type': ['origin']
    },
    geometry=[Point(sample_lon, sample_lat)],
    crs='EPSG:4326'
)

origin_path = os.path.join(OUTPUT_DIR, 'origin_stop.geojson')
origin_gdf.to_file(origin_path, driver='GeoJSON')
print(f"Exported: {origin_path}")


In [ ]:
# Summary of outputs
print("\n" + "="*60)
print("CORE ANALYSIS COMPLETE")
print("="*60)
print(f"\nOutputs saved to: {OUTPUT_DIR}")
print(f"\nGenerated files:")
for f in os.listdir(OUTPUT_DIR):
    fpath = os.path.join(OUTPUT_DIR, f)
    size = os.path.getsize(fpath) / 1024
    print(f"  - {f} ({size:.1f} KB)")

print(f"\nSample stop: {sample_stop['stop_name']}")
print(f"Isochrone distance: {MAX_DISTANCE}m")
print(f"Reachable nodes: {len(reachable_nodes)}")
